# Chapter 4: 分割数据和指令

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)


## 设置
运行以下设置单元格以加载您的 API 密钥并建立 get_completion 辅助函数。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

def get_completion(prompt: str, system_prompt=""):
    client = anthropic.Anthropic(api_key=API_KEY)
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---


## 课程
通常情况下，我们不想写完整的提示词，而是想要可以在提交给LLM之前用额外的输入数据进行修改的提示模板。如果您希望 LLM每次都做同样的事情，但 Claude 用于其任务的数据可能每次都不同，这可能会派上用场。
幸运的是，我们可以通过将提示的固定框架与可变用户输入分离，然后在将完整提示发送给 Claude 之前将用户输入替换到提示中来轻松实现这一点。
下面，我们将逐步介绍如何编写可替换的提示模板，以及如何替换用户输入。



## 示例
在第一个示例中，我们要求 LLM 充当动物声音生成器。请注意，提交给 LLM 的完整提示只是用输入（在这种情况下是"Cow"）替换的 PROMPT_TEMPLATE。请注意，当我们打印完整提示时，"Cow"这个词通过 f-string 替换了 ANIMAL 占位符。
注意： 在实践中，您不必将占位符变量命名为特定的名称。我们在这个示例中将其称为 ANIMAL，但同样容易地，我们也可以将其称为 CREATURE 或 A（尽管通常最好让您的变量名称具体且相关，这样即使没有替换，您的提示模板也易于理解，只是为了用户的可解析性）。只需确保您为变量命名的任何名称都是您在提示模板 f-string 中使用的名称。

In [78]:
# Variable content
动物 = "猪"

# Prompt template with a placeholder for the variable content
PROMPT = f"我会告诉您一个动物的名字。请用这个动物发出的声音来回应。. {动物}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
我会告诉您一个动物的名字。请用这个动物发出的声音来回应。. 猪

------------------------------------- Claude's response -------------------------------------
呼噜呼噜~




为什么我们要像这样分离和替换输入呢？嗯，**提示模板简化了重复性任务**。假设您构建了一个提示结构，邀请第三方用户向提示提交内容（在这种情况下是他们想要生成声音的动物）。这些第三方用户不必编写甚至看到完整的提示。他们所要做的就是填写变量。
我们在这里使用变量和 f-string 进行这种替换，但您也可以使用 format() 方法来实现。
**注意： 提示模板可以有任意数量的变量！**



当像这样引入替换变量时，确保 LLM **知道变量在哪里开始和结束（相对于指令或任务描述）非常重要**。让我们看一个指令和替换变量之间没有分离的示例。
对我们人类的眼睛来说，下面提示模板中变量的开始和结束位置非常清楚。然而，在完全替换的提示中，这种界限变得不明确。

In [ ]:
# Variable content
EMAIL = "我要求你明天早晨6点到达，因为我是公司CEO."

# Prompt template with a placeholder for the variable content
PROMPT = f"You Jack. {EMAIL} <----- 让EMAL的内容更加礼貌，但不要改变其他任何内容."


# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- LLM's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
You Jack. 我要求你明天早晨6点到达，因为我是公司CEO. <----- 让EMAL的内容更加礼貌，但不要改变其他任何内容.

------------------------------------- LLM's response -------------------------------------
亲爱的CEO,

我很高兴能为公司效劳。我会尽力在明天早上6点准时到达。我理解您作为公司领导的重要性,我会全力配合您的要求。

如果有任何需要我协助的地方,请随时告诉我。我会竭尽全力完成任务,为公司做出应有的贡献。

再次感谢您的信任,我会在明天早上准时到达。

祝工作顺利!


Here, **Claude thinks "Yo Claude" is part of the email it's supposed to rewrite**! You can tell because it begins its rewrite with "Dear Claude". To the human eye, it's clear, particularly in the prompt template where the email begins and ends, but it becomes much less clear in the prompt after substitution.

在这里，**LLM 认为"Yo Claude"是它应该重写的邮件的一部分！** 您可以通过它用"Dear Claude"开始重写来看出这一点。对人类的眼睛来说，特别是在提示模板中，邮件的开始和结束位置很清楚，但在替换后的提示中变得不那么清楚了。


我们如何解决这个问题？**用 XML 标签包装输入**！我们在下面这样做了，正如您所看到的，输出中不再有"Dear Claude"。
XML 标签是像 <tag></tag> 这样的尖括号标签。它们成对出现，由开始标签（如 <tag>）和由 / 标记的结束标签（如 </tag>）组成。XML 标签用于包装内容，像这样：<tag>content</tag>。
注意： 虽然 Claude 可以识别和使用各种分隔符和定界符，但我们建议您专门使用 XML 标签作为 LLM 的分隔符，因为 LLM 经过专门训练，将 XML 标签识别为提示组织机制。除了函数调用之外，没有 LLM 经过训练的特殊 XML 标签可以让您最大限度地提升性能。我们有意让 LLM 以这种方式变得非常灵活和可定制。

In [75]:
# Variable content
EMAL = "我要求你明天早晨6点到达，因为我是公司CEO."

# Prompt template with a placeholder for the variable content
PROMPT = f"You Jack. <EMAL> {EMAL} </EMAL>  <-----让EMAIL的内容更加礼貌，但不要改变其他任何内容."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))


让我们看看 XML 标签如何帮助我们的另一个示例。
在以下提示中，**LLM错误地解释了提示的哪一部分是指令与输入**。由于格式的原因，它错误地将 Each is about an animal, like rabbits 视为列表的一部分，而用户（填写 SENTENCES 变量的人）可能不希望这样。

In [56]:
# Variable content

SENTENCES = """- 我喜欢牛的叫声
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# Prompt template with a placeholder for the variable content

PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

-每个都是关于动物的，比如兔子。
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

要修复这个问题，我们只需要用 **XML 标签包围用户输入的句子**。这向 LLM 显示了输入数据的开始和结束位置，尽管在 Each is about an animal, like rabbits. 之前有误导性的连字符`

In [58]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""


SENTENCES = """- 我喜欢牛的叫声
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# Prompt template with a placeholder for the variable content

PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。
-每个都是关于动物的，比如兔子。

<sentences>
{SENTENCES}
</sentences>"""



# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
下面是一个句子列表。告诉我列表中的第二项。
-每个都是关于动物的，比如兔子。

<sentences>
- 我喜欢牛的叫声
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的
</sentences>

------------------------------------- Claude's response -------------------------------------
这句话是关于蜘蛛的。


注意： 在"Each is about an animal"提示的错误版本中，我们必须包含连字符才能让 LLM 以我们为这个示例想要的方式错误地回应。这是关于提示工程的重要教训：**小细节很重要！** ，**清理提示中的拼写错误和语法错误总是值得的**。LLM 对模式很敏感（在早期，在微调之前，它是一个原始的文本预测工具），当您犯错时它更可能犯错，当您听起来聪明时它更聪明，当您听起来愚蠢时它更愚蠢，等等。


---

## 练习
- [Exercise 4.1 - Haiku Topic](#exercise-41---haiku-topic)
- [Exercise 4.2 - Dog Question with Typos](#exercise-42---dog-question-with-typos)
- [Exercise 4.3 - Dog Question Part 2](#exercise-42---dog-question-part-2)


### 练习 4.1 - 诗歌主题
修改 PROMPT，使其成为一个模板，该模板将接收一个名为 TOPIC 的变量，并输出关于该主题的诗歌。这个练习只是为了测试您对使用 f-string 的变量模板结构的理解。

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("诗歌", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!


### 练习 4.2 - 带有拼写错误的狗问题
通过添加 XML 标签来修复 PROMPT，使 LLM 产生正确的答案。
尽量不要改变提示的其他任何内容。混乱和错误百出的写作是故意的，这样您可以看到 LLM 对这些错误的反应

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))


### 练习 4.3 - 狗问题第二部分

修复 PROMPT，不要添加 XML 标签。相反，只从提示中删除一两个单词。
与上述练习一样，尽量不要改变提示的其他任何内容。这将向您展示 LLM 能够解析和理解什么样的语言。

In [89]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
Hia its me i have a q about dogs jkaerjv ar cn brown? jklmvca tx it help me muhch much atx fst fst answer short short tx

------------------------------------- Claude's response -------------------------------------
I'm afraid I don't fully understand your question. Could you please rephrase it in a more clear and concise way? I'd be happy to try to answer your question about dogs if you can provide it in a more straightforward manner.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: False




### 恭喜！
如果您已经解决了到此为止的所有练习，您就可以进入下一章了。祝您提示词工程愉快！